In [ ]:
import cv2
import torch
import pandas as pd
from collections import defaultdict
from ultralytics import YOLO
import supervision as sv

In [ ]:
YOLO_WEIGHT = 'yolov8n.pt'
CUSTOM_WEIGHT = 'runs/detect/train/weights/best.pt'
TEST_VIDEO = 'sample2.mp4'

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.cuda.is_available()
model = YOLO(YOLO_WEIGHT)

In [ ]:
# Supervision as frame generator
generator = sv.get_video_frames_generator(TEST_VIDEO)
count = 1
model = YOLO(YOLO_WEIGHT)
classNames = model.names
annotator = sv.BoxAnnotator(thickness=2, text_thickness=1, text_scale=1)
d = []
track_history = defaultdict(lambda: [])
video_info = sv.VideoInfo.from_video_path(TEST_VIDEO)
 
with sv.VideoSink(target_path='result.mp4', video_info=video_info) as sink:
    for frame in generator:
        
        count += 1
        results = model.track(
            source=frame,
            persist=True,
            stream=True,
            agnostic_nms=True,
            device=device
        )

        for r in results:
            annotated_frame = r.plot()

            boxes = r.boxes.cpu().numpy()
            xyxys = boxes.xyxy

            labels = [
                f'#{count} {xyxys}'
            ]

            detections = sv.Detections.from_ultralytics(r)
            detections = detections[detections.class_id == 0]
            annotator = sv.BoxAnnotator()
            annotated_frame = annotator.annotate(
                scene=frame.copy(),
                detections=detections,
                labels=labels
            )

            d.append(
                 {
                      'frame': count,
                      'xyxy': xyxys
                 }
            )

        sink.write_frame(frame=annotated_frame)
        cv2.imshow("YOLOv8", annotated_frame)
        
        df = pd.DataFrame(d)
        df.to_csv('result.csv')

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cv2.destroyAllWindows()